# Docling PDF Reader example

In [ ]:
from rich.pretty import pprint
import warnings
import os
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Basic usage

By default, `DoclingPDFReader` exports to Markdown. Basic usage looks like this:

In [ ]:
from llama_index.readers.docling.base import DoclingPDFReader

reader = DoclingPDFReader()
docs = reader.load_data(file_path="https://arxiv.org/pdf/2408.09869")

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Previewing a sample of the loaded doc's Markdown text:

In [ ]:
import rich

md_snippet = docs[0].text[:700]
rich.print(f"{md_snippet}...")

<!-- image -->

## Docling Technical Report

## Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis 
Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima 
Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research Ruschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF 
document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and 
table structure recognition (TableFormer), and run...

## RAG demo

Setting up the node parser:

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser()

Getting the nodes (chunks) and displaying a sample:

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents=docs)
pprint(nodes[7], max_length=2, max_string=250, max_depth=2)

TextNode(
│   id_='5ba8335e-4ea7-4c24-ac16-f2b85883417a',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'Header_2': '3.1 PDF backends'
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash'],
│   excluded_llm_metadata_keys=['dl_doc_hash'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   ... +1
│   },
│   text='3.1 PDF backends\n\nTwo basic requirements to process PDF documents in our pipeline are a) to retrieve all text content and their geometric coordinates on each page and b) to render the visual representation of each page as it would appear in a PDF vie'+1029,
│   mimetype='text/plain',
│   start_char_idx=5009,
│   end_char_idx=6288,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

Setting up the embed model:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Setting up the vector store:

In [ ]:
from tempfile import TemporaryDirectory
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri=os.environ.get(
        "MILVUS_URL", default=f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
    ),
    collection_name="docling_collection",
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Setting up the index:

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embed_model,
    storage_context=storage_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/33 [00:00<?, ?it/s]

Setting up the LLM:

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN = os.environ.get("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
)

/Users/pva/work/github.com/vagenas/llama_index/llama-index-integrations/readers/llama-index-readers-docling/.venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


And now we are ready to perform questions on our document content:

In [ ]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=10)
query_res = query_engine.query("How many pages were human annotated?")
pprint(query_res, max_length=1, max_string=70, max_depth=4)

Response(
│   response='80863 pages were human annotated.',
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='798e2f7e-d310-4831-be50-01be29faf7a0',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysi'+365,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=20310,
│   │   │   │   end_char_idx=20745,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7126335501670837
│   │   ),
│   │   ... +7
│   ],
│   metadata={
│   │   '798e2f7e-d310-4831-be50-01be29faf7a0': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   ... +1
│   │   },
│   │   ... +7
│   }
)